# Part A: GC Content and Sequence Analysis

## Question 1: GC Content Calculation

In [4]:
%pip install numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 1.4 MB/s eta 0:00:00a 0:00:010m
Note: you may need to restart the kernel to use updated packages.


In [5]:
import collections
import numpy as np

def calculate_overall_gc_content(sequence: str) -> float:
    """
    Calculates the overall GC content percentage of a DNA sequence[cite: 26].

    :param sequence: The input DNA sequence string.
    :return: The GC content percentage (float).
    """
    sequence = sequence.upper().replace(' ', '')
    gc_count = sequence.count('G') + sequence.count('C')
    total_length = len(sequence)
    if total_length == 0:
        return 0.0
    return (gc_count / total_length) * 100.0

def gc_content_sliding_window(sequence: str, window_size: int = 100, step: int = 50) -> list:
    """
    Analyzes GC content in sliding windows[cite: 27].

    :param sequence: The input DNA sequence string.
    :param window_size: The size of the sliding window.
    :param step: The step size for the window.
    :return: A list of GC content percentages for each window.
    """
    sequence = sequence.upper().replace(' ', '')
    gc_contents = []
    
    for i in range(0, len(sequence) - window_size + 1, step):
        window = sequence[i:i + window_size]
        gc_content = calculate_overall_gc_content(window)
        gc_contents.append(gc_content)
        
    return gc_contents

def generate_gc_skew(sequence: str) -> dict:
    """
    Generates GC skew statistics (G-C)/(G+C) in a cumulative manner[cite: 28].

    :param sequence: The input DNA sequence string.
    :return: A dictionary containing the list of cumulative GC skew values.
    """
    sequence = sequence.upper().replace(' ', '')
    cumulative_skew = [0]
    g_count = 0
    c_count = 0

    for base in sequence:
        if base == 'G':
            g_count += 1
        elif base == 'C':
            c_count += 1
        
        # Calculate skew for the current window (G-C) / (G+C)
        gc_sum = g_count + c_count
        skew = (g_count - c_count) / gc_sum if gc_sum > 0 else 0
        cumulative_skew.append(skew)

    # Remove the initial zero
    return {"cumulative_gc_skew": cumulative_skew[1:]} 

# Note: Comparison across species (Q1, Part 4) requires external data, 
# which is handled by applying the above functions to a list of species sequences.

## Question 2: Sequence Composition Analysis

In [6]:
def calculate_kmer_frequencies(sequence: str, k: int) -> collections.Counter:
    """
    Calculates k-mer frequencies (e.g., dinucleotide/trinucleotide)[cite: 31].

    :param sequence: The input DNA sequence string.
    :param k: The size of the k-mer (e.g., 2 for dinucleotide, 3 for trinucleotide).
    :return: A Counter object with k-mer counts.
    """
    sequence = sequence.upper().replace(' ', '')
    kmers = [sequence[i:i+k] for i in range(len(sequence) - k + 1)]
    return collections.Counter(kmers)

def identify_cpg_islands(sequence: str, min_length: int = 200, min_gc: float = 50.0, min_ratio: float = 0.6) -> list:
    """
    Identifies potential CpG islands based on length, GC content, and CpG observed/expected ratio[cite: 32].

    :param sequence: The input DNA sequence string.
    :param min_length: Minimum length of a sequence region to be considered an island.
    :param min_gc: Minimum GC content percentage for an island.
    :param min_ratio: Minimum CpG observed/expected ratio for an island.
    :return: A list of tuples (start_index, end_index, CpG_ratio, GC_content) for identified islands.
    """
    sequence = sequence.upper().replace(' ', '')
    cpg_islands = []
    
    # Simple sliding window check (e.g., window size = 100, step = 1)
    # A more rigorous implementation would use dynamic programming or boundary detection.
    window_size = min_length
    
    for i in range(len(sequence) - window_size + 1):
        window = sequence[i:i + window_size]
        
        # 1. GC Content Check
        gc_content = calculate_overall_gc_content(window)
        if gc_content < min_gc:
            continue
            
        # 2. CpG Observed/Expected Ratio Check
        c_count = window.count('C')
        g_count = window.count('G')
        cpg_count = window.count('CG')
        
        expected_cpg = (c_count * g_count) / window_size
        
        if expected_cpg == 0:
            cpg_ratio = 0.0
        else:
            cpg_ratio = cpg_count / expected_cpg
        
        if cpg_ratio >= min_ratio:
            cpg_islands.append((i, i + window_size - 1, round(cpg_ratio, 3), round(gc_content, 2)))
            
    return cpg_islands

def analyze_codon_usage_bias(protein_coding_sequence: str, genetic_code: dict) -> dict:
    """
    Analyzes codon usage bias by calculating the frequency of each codon for each amino acid[cite: 33].

    :param protein_coding_sequence: The DNA/RNA sequence of the coding region.
    :param genetic_code: A dictionary mapping codons to amino acids.
    :return: A dictionary where keys are amino acids and values are dictionaries of codon counts and frequencies.
    """
    codons = [protein_coding_sequence[i:i+3] for i in range(0, len(protein_coding_sequence) - 2, 3)]
    codon_counts = collections.Counter(codons)
    
    amino_acid_usage = collections.defaultdict(lambda: collections.defaultdict(int))
    
    for codon, count in codon_counts.items():
        if codon in genetic_code:
            aa = genetic_code[codon]
            amino_acid_usage[aa]['counts'][codon] = count
            amino_acid_usage[aa]['total_aa_count'] += count
            
    # Calculate frequencies
    bias_report = {}
    for aa, data in amino_acid_usage.items():
        total = data['total_aa_count']
        bias_report[aa] = {
            codon: (count / total) if total > 0 else 0.0 
            for codon, count in data['counts'].items()
        }

    return dict(bias_report)

# Note: Sequence classification (Q2, Part 4) is a machine learning/statistical task 
# that uses k-mer frequencies as features; the calculation is provided by 
# `calculate_kmer_frequencies`.

# Part B: Pattern Matching and Motif Discovery

## Question 3: Hamming Distance Implementation

In [7]:
import numpy as np
from typing import List

def hamming_distance(seq1: str, seq2: str) -> int:
    """
    Implements Hamming distance for sequence comparison[cite: 37].
    The Hamming distance is only defined for sequences of equal length.

    :param seq1: The first DNA sequence string.
    :param seq2: The second DNA sequence string.
    :raises ValueError: If sequences are of different lengths.
    :return: The number of positions at which the corresponding symbols are different.
    """
    seq1 = seq1.upper().replace(' ', '')
    seq2 = seq2.upper().replace(' ', '')

    if len(seq1) != len(seq2):
        # Handles sequences of different lengths by raising an error (standard definition) [cite: 38]
        raise ValueError("Hamming distance is only defined for sequences of equal length.")

    distance = sum(c1 != c2 for c1, c2 in zip(seq1, seq2))
    return distance

def calculate_distance_matrix(sequences: List[str]) -> np.ndarray:
    """
    Calculates the distance matrix for multiple sequences using Hamming distance[cite: 39].
    Only compares sequences of the same length, skipping unequal pairs.

    :param sequences: A list of DNA sequence strings.
    :return: A square NumPy array (distance matrix).
    """
    n = len(sequences)
    # Initialize matrix with -1 to indicate invalid/unequal length comparisons
    distance_matrix = np.full((n, n), -1, dtype=int) 

    for i in range(n):
        for j in range(i, n):
            s1 = sequences[i].upper().replace(' ', '')
            s2 = sequences[j].upper().replace(' ', '')
            
            if len(s1) == len(s2):
                dist = hamming_distance(s1, s2)
                distance_matrix[i, j] = dist
                distance_matrix[j, i] = dist # Symmetric matrix
            # else: distance remains -1 (or other indicator of non-comparable)

    return distance_matrix

# Note: Visualization (Q3, Part 4) is typically done externally with libraries 
# like Matplotlib (for heatmaps) or SciPy/SciKit-learn (for clustering/trees 
# based on the distance matrix). The matrix is the appropriate data structure[cite: 40].

## Question 4: Motif Finding Algorithms

In [8]:
def find_exact_pattern(text: str, pattern: str) -> List[int]:
    """
    Implements exact pattern matching algorithms (simple iteration)[cite: 42].
    For efficiency, advanced algorithms like KMP or Boyer-Moore could be used.

    :param text: The genomic sequence to search within.
    :param pattern: The exact pattern (motif) to find.
    :return: A list of starting indices where the pattern occurs.
    """
    text = text.upper()
    pattern = pattern.upper()
    indices = []
    n, m = len(text), len(pattern)
    
    if m == 0 or n < m:
        return []
        
    # Standard O(N*M) naive search
    for i in range(n - m + 1):
        if text[i:i+m] == pattern:
            indices.append(i)
            
    return indices

def find_fuzzy_pattern(text: str, pattern: str, max_mismatches: int) -> List[int]:
    """
    Creates fuzzy matching with allowed mismatches (Hamming distance)[cite: 43].

    :param text: The genomic sequence to search within.
    :param pattern: The pattern (motif) to find.
    :param max_mismatches: The maximum number of allowed mismatches.
    :return: A list of starting indices where the pattern occurs with <= max_mismatches.
    """
    text = text.upper()
    pattern = pattern.upper()
    indices = []
    n, m = len(text), len(pattern)
    
    if m == 0 or n < m:
        return []

    # O(N*M) check for mismatches
    for i in range(n - m + 1):
        substring = text[i:i+m]
        mismatches = 0
        
        # Calculate Hamming distance between pattern and substring
        for j in range(m):
            if pattern[j] != substring[j]:
                mismatches += 1
                if mismatches > max_mismatches:
                    break
                    
        if mismatches <= max_mismatches:
            indices.append(i)
            
    return indices

def find_all_occurrences(text: str, patterns: List[str], max_mismatches: int = 0) -> dict:
    """
    Finds all occurrences of multiple patterns in a genomic sequence[cite: 44].

    :param text: The genomic sequence to search within.
    :param patterns: A list of patterns (motifs) to find.
    :param max_mismatches: The maximum number of allowed mismatches (0 for exact).
    :return: A dictionary mapping each pattern to a list of its starting indices.
    """
    results = {}
    for pattern in patterns:
        if max_mismatches == 0:
            results[pattern] = find_exact_pattern(text, pattern)
        else:
            results[pattern] = find_fuzzy_pattern(text, pattern, max_mismatches)
            
    return results

# Note: Motif conservation analysis (Q4, Part 45) is typically done by running 
# `find_fuzzy_pattern` on multiple sequences with the same motif and comparing 
# the resulting match scores/mismatches.

# Part C: Protein Sequence Analysis

## Question 5: Reading Frame Analysis

In [9]:
def find_all_reading_frames(dna_sequence: str) -> dict:
    """
    Identifies all six reading frames in a DNA sequence[cite: 48].

    :param dna_sequence: The input DNA sequence string.
    :return: A dictionary mapping frame number (1 to 6) to the sequence in that frame.
    """
    from .assignment1_q5 import get_reverse_complement # Reuses Q5 from Assignment 1
    
    dna_sequence = dna_sequence.upper().replace(' ', '')
    frames = {}
    
    # Forward frames (1, 2, 3)
    for i in range(3):
        frames[i + 1] = dna_sequence[i:]
        
    # Reverse frames (4, 5, 6) are derived from the reverse complement
    reverse_complement = get_reverse_complement(dna_sequence)
    for i in range(3):
        frames[i + 4] = reverse_complement[i:]
        
    return frames

def find_orfs_in_frame(frame_sequence: str, genetic_code: dict) -> List[str]:
    """
    Finds start and stop codons and extracts potential protein sequences (ORFs)[cite: 49, 50].

    :param frame_sequence: The DNA/RNA sequence for one reading frame.
    :param genetic_code: A dictionary mapping codons to amino acids (used for start/stop).
    :return: A list of potential protein sequences (amino acid strings).
    """
    
    # Use standard ATG (DNA) or AUG (RNA) as START and TAA/TAG/TGA (DNA) or UAA/UAG/UGA (RNA) as STOP
    # This implementation assumes DNA input.
    START_CODONS = {"ATG"}
    STOP_CODONS = {"TAA", "TAG", "TGA"}
    
    orfs = []
    i = 0
    n = len(frame_sequence)
    in_orf = False
    current_orf_dna = ""
    
    while i < n - 2:
        codon = frame_sequence[i:i+3].upper()
        
        if not in_orf:
            if codon in START_CODONS:
                in_orf = True
                current_orf_dna = codon # Start collecting DNA sequence
        
        elif in_orf:
            if codon in STOP_CODONS:
                in_orf = False
                # Translate the DNA sequence collected in the ORF
                protein_sequence = translate_dna_to_protein(current_orf_dna, genetic_code)
                orfs.append(protein_sequence)
                current_orf_dna = ""
            else:
                current_orf_dna += codon
                
        i += 3 # Move to the next codon

    # Calculate ORF statistics (Q5, Part 51) is done by analyzing the lengths of the list elements
    # `[len(orf) for orf in orfs]` and generating standard statistics (mean, median, max).
    return orfs

## Question 6: Protein Translation and Analysis

In [10]:
# Genetic code dictionary for DNA to Amino Acid translation [cite: 54]
# Simplified code (full version is much larger and includes all 64 codons)
GENETIC_CODE = {
    'TTT': 'F', 'TTC': 'F', 'TTA': 'L', 'TTG': 'L',
    'TCT': 'S', 'TCC': 'S', 'TCA': 'S', 'TCG': 'S',
    'TAT': 'Y', 'TAC': 'Y', 'TAA': '*', 'TAG': '*', # * = STOP
    'TGT': 'C', 'TGC': 'C', 'TGA': '*', 'TGG': 'W',
    'CTT': 'L', 'CTC': 'L', 'CTA': 'L', 'CTG': 'L',
    'CCT': 'P', 'CCC': 'P', 'CCA': 'P', 'CCG': 'P',
    'CAT': 'H', 'CAC': 'H', 'CAA': 'Q', 'CAG': 'Q',
    'CGT': 'R', 'CGC': 'R', 'CGA': 'R', 'CGG': 'R',
    'ATT': 'I', 'ATC': 'I', 'ATA': 'I', 'ATG': 'M', # ATG = START/Methionine
    'ACT': 'T', 'ACC': 'T', 'ACA': 'T', 'ACG': 'T',
    'AAT': 'N', 'AAC': 'N', 'AAA': 'K', 'AAG': 'K',
    'AGT': 'S', 'AGC': 'S', 'AGA': 'R', 'AGG': 'R',
    'GTT': 'V', 'GTC': 'V', 'GTA': 'V', 'GTG': 'V',
    'GCT': 'A', 'GCC': 'A', 'GCA': 'A', 'GCG': 'A',
    'GAT': 'D', 'GAC': 'D', 'GAA': 'E', 'GAG': 'E',
    'GGT': 'G', 'GGC': 'G', 'GGA': 'G', 'GGG': 'G',
}


def translate_dna_to_protein(sequence: str, genetic_code: dict) -> str:
    """
    Converts a DNA sequence (in one reading frame) to an amino acid sequence[cite: 53].

    :param sequence: The DNA sequence string (must be a multiple of 3).
    :param genetic_code: The genetic code dictionary.
    :return: The translated amino acid sequence string.
    """
    sequence = sequence.upper().replace(' ', '')
    amino_acids = []
    
    for i in range(0, len(sequence) - 2, 3):
        codon = sequence[i:i+3]
        amino_acid = genetic_code.get(codon, 'X') # 'X' for unknown/invalid codon
        amino_acids.append(amino_acid)
        
    return "".join(amino_acids)

def translate_longest_orfs(dna_sequence: str, genetic_code: dict) -> List[str]:
    """
    Identifies and translates the longest ORFs across all six reading frames[cite: 55].

    :param dna_sequence: The DNA sequence string.
    :param genetic_code: The genetic code dictionary.
    :return: A list containing the longest protein sequence from each frame (can contain duplicates).
    """
    frames = find_all_reading_frames(dna_sequence)
    longest_orfs_per_frame = []
    
    for frame_num, frame_seq in frames.items():
        all_orfs = find_orfs_in_frame(frame_seq, genetic_code)
        
        if all_orfs:
            # Find the longest ORF in the current frame
            longest_orf = max(all_orfs, key=len)
            longest_orfs_per_frame.append(longest_orf)
            
    return longest_orfs_per_frame

def analyze_amino_acid_composition(protein_sequence: str) -> dict:
    """
    Analyzes amino acid composition and properties[cite: 56].
    
    :param protein_sequence: The translated amino acid sequence.
    :return: A dictionary containing amino acid counts and basic stats (e.g., length).
    """
    protein_sequence = protein_sequence.upper().replace(' ', '').replace('*', '') # Remove stop codons
    
    # 1. Composition (Counts)
    aa_counts = collections.Counter(protein_sequence)
    
    # 2. Basic Properties/Statistics
    total_length = len(protein_sequence)
    
    # A simple property: Hydrophobicity score (arbitrary values for demonstration)
    HYDROPHOBICITY = {'A': 1.8, 'G': -0.4, 'I': 4.5, 'L': 3.8, 'V': 4.2, 
                      'F': 2.8, 'P': -1.6, 'Y': -1.3, 'M': 1.9, 'T': -0.7, 
                      'W': -0.9, 'C': 2.5, 'K': -3.9, 'R': -4.5, 'H': -3.2, 
                      'E': -3.5, 'D': -3.5, 'N': -3.5, 'Q': -3.5, 'S': -0.8}
    
    hydrophobicity_score = sum(HYDROPHOBICITY.get(aa, 0.0) for aa in protein_sequence)
    avg_hydrophobicity = hydrophobicity_score / total_length if total_length else 0.0

    analysis = {
        'length': total_length,
        'amino_acid_counts': dict(aa_counts),
        'avg_hydrophobicity_score': round(avg_hydrophobicity, 3),
        # More properties like charge, molecular weight can be added here
    }
    return analysis